In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from typing import Dict, Text
import tensorflow_recommenders as tfrs

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
carts_df = pd.read_csv('all_carts.csv', index_col=0)
items_df = pd.read_csv('all_libelles.csv', index_col=0)

In [7]:
carts = tf.data.Dataset.from_tensor_slices(dict(carts_df))
carts = carts.shuffle(buffer_size=50)
carts = carts.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

items = tf.data.Dataset.from_tensor_slices(dict(items_df))
items = items.shuffle(buffer_size=50)
items = items.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [10]:
cart_map = carts.map(lambda x: x['TICKET_ID'])
item_map = items.map(lambda x: x['LIBELLE'])

In [11]:
cart_id_vocabulary = tf.keras.layers.experimental.preprocessing.IntegerLookup(mask_token=None)
cart_id_vocabulary.adapt(cart_map)

In [12]:
libelles_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
libelles_vocabulary.adapt(item_map)

In [13]:
items_values = items.map(lambda x: x['LIBELLE'])
carts_values = carts.map(lambda x:{'LIBELLE':x['LIBELLE'],'TICKET_ID':x['TICKET_ID']})

In [14]:
class LibelleRecModel(tfrs.Model):
 
  def __init__(
      self,
      cart_model: tf.keras.Model,
      libelle_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()
 
    # Set up user and movie representations.
    self.cart_model = cart_model
    self.libelle_model = libelle_model
 
    # Set up a retrieval task.
    self.task = task
 
  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.
 
    cart_embeddings = self.cart_model(features["TICKET_ID"])
    libelle_embeddings = self.libelle_model(features["LIBELLE"])
 
    return self.task(cart_embeddings, libelle_embeddings)

In [15]:
carts_model = tf.keras.Sequential([cart_id_vocabulary,tf.keras.layers.Embedding(cart_id_vocabulary.vocabulary_size(),64)])
libelles_model = tf.keras.Sequential([libelles_vocabulary,tf.keras.layers.Embedding(libelles_vocabulary.vocabulary_size(),64)])
 
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(items_values.batch(128).map(libelles_model)))

In [17]:
carts_model.save("cm")
libelles_model.save("lm")

INFO:tensorflow:Assets written to: cm\assets
INFO:tensorflow:Assets written to: lm\assets


In [16]:
model = LibelleRecModel(carts_model,libelles_model,task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))
model.fit(carts_values.batch(4096), epochs=3)

Epoch 1/3
  89/1769 [>.............................] - ETA: 141:24:36 - factorized_top_k/top_1_categorical_accuracy: 2.7706e-04 - factorized_top_k/top_5_categorical_accuracy: 2.7706e-04 - factorized_top_k/top_10_categorical_accuracy: 2.7706e-04 - factorized_top_k/top_50_categorical_accuracy: 2.7706e-04 - factorized_top_k/top_100_categorical_accuracy: 2.7706e-04 - loss: 34114.5895 - regularization_loss: 0.0000e+00 - total_loss: 34114.5895

KeyboardInterrupt: 

In [ ]:
recommends = tfrs.layers.factorized_top_k.BruteForce(model.cart_model)
recommends.index_from_dataset(items_values.batch(100).map(lambda libelle: (libelle, model.libelle_model(libelle))))